In [ ]:
 !wget https://github.com/adoptium/temurin11-binaries/releases/download/jdk-11.0.24%2B8/OpenJDK11U-jdk_x64_linux_hotspot_11.0.24_8.tar.gz
!tar xzf OpenJDK11U-jdk_x64_linux_hotspot_11.0.24_8.tar.gz
!rm OpenJDK11U-jdk_x64_linux_hotspot_11.0.24_8.tar.gz
!wget https://archive.apache.org/dist/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xzf spark-3.5.2-bin-hadoop3.tgz
!rm spark-3.5.2-bin-hadoop3.tgz
!pip install findspark

--2025-08-04 01:46:33--  https://github.com/adoptium/temurin11-binaries/releases/download/jdk-11.0.24%2B8/OpenJDK11U-jdk_x64_linux_hotspot_11.0.24_8.tar.gz
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/372924883/a16cd959-0718-43a7-a5f5-829aa52ef267?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-04T02%3A44%3A07Z&rscd=attachment%3B+filename%3DOpenJDK11U-jdk_x64_linux_hotspot_11.0.24_8.tar.gz&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-04T01%3A43%3A14Z&ske=2025-08-04T02%3A44%3A07Z&sks=b&skv=2018-11-09&sig=ln1XG0lAuPq4ue3lxMlwoaJ60Ym984tJCC68HEtCDhY%3D&jwt=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4c

In [ ]:
import os
import sys
import subprocess

working_dir = subprocess.run(['pwd'], stdout = subprocess.PIPE).stdout.strip().decode("utf-8")
print(working_dir)
os.environ["JAVA_HOME"] = working_dir + "/jdk-11.0.24+8/"
os.environ["SPARK_HOME"] = working_dir + "/spark-3.5.2-bin-hadoop3/"
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9.7-src.zip")

number_cores = 2
memory_gb = 6

/content


In [ ]:
import findspark
import pyspark

findspark.init()

conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))

sc = pyspark.SparkContext(conf=conf)

In [ ]:
# Installs pybaseball for Statcast data
!pip install pybaseball --quiet

# Imports Statcast scraper
from pybaseball import statcast
import pandas as pd

print("Downloading 2022 data...")
data_2022 = statcast(start_dt="2022-04-07", end_dt="2022-10-02")

print("Downloading 2023 data...")
data_2023 = statcast(start_dt="2023-03-30", end_dt="2023-10-01")

print("Downloading 2024 data... ")
data_2024 = statcast(start_dt="2024-03-28", end_dt="2024-09-29")

combined = pd.concat([data_2022, data_2023, data_2024])

# Saving the data to a CSV may take time (took me around 13 minutes))
combined.to_csv("statcast_2022_2024.csv", index=False)
print("Saved as statcast_2022_2024.csv")

# Preview row count (Spoiler alert: it should be 2,121,390)
print(f"Total rows: {combined.shape[0]}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.5/416.5 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 41.4 MB/s eta 0:00:00
This is a large query, it may take a moment to complete


/usr/local/lib/python3.11/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
  0%|          | 0/179 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_co

This is a large query, it may take a moment to complete


/usr/local/lib/python3.11/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
  0%|          | 0/186 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_co

This is a large query, it may take a moment to complete


/usr/local/lib/python3.11/dist-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
  0%|          | 0/186 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/pybaseball/datahelpers/postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_co

Saved as statcast_2022_2024.csv
Total rows: 2121390


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg, count

spark = SparkSession(sc)

df = spark.read.option("header", "true").option("inferSchema", "true").csv("statcast_2022_2024.csv")
df.printSchema()
print(f"Total rows: {df.count()}")
df.show(5)

root
 |-- pitch_type: string (nullable = true)
 |-- game_date: date (nullable = true)
 |-- release_speed: double (nullable = true)
 |-- release_pos_x: double (nullable = true)
 |-- release_pos_z: double (nullable = true)
 |-- player_name: string (nullable = true)
 |-- batter: integer (nullable = true)
 |-- pitcher: integer (nullable = true)
 |-- events: string (nullable = true)
 |-- description: string (nullable = true)
 |-- spin_dir: string (nullable = true)
 |-- spin_rate_deprecated: string (nullable = true)
 |-- break_angle_deprecated: string (nullable = true)
 |-- break_length_deprecated: string (nullable = true)
 |-- zone: integer (nullable = true)
 |-- des: string (nullable = true)
 |-- game_type: string (nullable = true)
 |-- stand: string (nullable = true)
 |-- p_throws: string (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- type: string (nullable = true)
 |-- hit_location: integer (nullable = true)
 |-- bb_type: string (

In [ ]:
df_cleaned = df.drop("player_name")

In [ ]:
from pybaseball import playerid_reverse_lookup
import pandas as pd

# Gets unique batter IDs
batter_ids = df_cleaned.select("batter").distinct().toPandas()
batter_ids = batter_ids["batter"].dropna().astype(int).tolist()

# Reverse lookup to get names
lookup_df = playerid_reverse_lookup(batter_ids, key_type='mlbam')
lookup_df['batter'] = lookup_df['key_mlbam']
lookup_df['player_name'] = lookup_df['name_first'] + ' ' + lookup_df['name_last']

# Renames player_name
lookup_df = lookup_df.rename(columns={"player_name": "batter_name"})

lookup_df = lookup_df[['batter', 'batter_name']]

Gathering player lookup table. This may take a moment.


In [ ]:
lookup_spark_df = spark.createDataFrame(lookup_df)
df_with_names = df.drop("player_name").join(lookup_spark_df, on="batter", how="left")

## Filter and Aggregate Hitter Statistics

In this step, I filtered for batted balls with complete tracking data: exit velocity (launch_speed), launch angle (launch_angle), and estimated hit distance (hit_distance_sc). Then I grouped by batter and calculated the average of each metric. To reduce noise from small sample sizes, I also filtered out hitters with fewer than 50 batted ball events.


In [ ]:
batted_balls = df_with_names.filter(
    (col("launch_speed").isNotNull()) &
    (col("launch_angle").isNotNull()) &
    (col("events").isNotNull())
)

hitter_stats = (
    batted_balls.groupBy("batter", "batter_name")
    .agg(
        avg("launch_speed").alias("avg_exit_velocity"),
        avg("launch_angle").alias("avg_launch_angle"),
        avg("hit_distance_sc").alias("avg_hit_distance"),
        count("*").alias("num_batted_balls")
    )
    .filter(col("num_batted_balls") >= 50)
)

## Vectorization, Scaling, and K-Means Clustering

I used Spark's MLlib to build a pipeline that performs the following:
1. Combines the hitting stats into a feature vector
2. Standardizes the features using StandardScaler
3. Applies K-Means clustering with k = 4 to segment hitters into groups based on their hitting profiles

In [ ]:
from pyspark.ml.feature import VectorAssembler, StandardScaler

assembler = VectorAssembler(
    inputCols=["avg_exit_velocity", "avg_launch_angle", "avg_hit_distance"],
    outputCol="features"
)

scaler = StandardScaler(
    inputCol="features",
    outputCol="scaled_features",
    withMean=True,
    withStd=True
)

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline

kmeans = KMeans(featuresCol="scaled_features", k=4, seed=42, predictionCol="cluster")

pipeline = Pipeline(stages=[assembler, scaler, kmeans])
model = pipeline.fit(hitter_stats)

clustered_data = model.transform(hitter_stats).select(
    "batter_name", "avg_exit_velocity", "avg_launch_angle",
    "avg_hit_distance", "num_batted_balls", "cluster"
)

clustered_data.show(20)

+------------------+-----------------+------------------+------------------+----------------+-------+
|       batter_name|avg_exit_velocity|  avg_launch_angle|  avg_hit_distance|num_batted_balls|cluster|
+------------------+-----------------+------------------+------------------+----------------+-------+
|   yasmani grandal|89.15042979942692|11.091690544412607|159.23638968481376|             698|      1|
|      yoán moncada|88.85538752362955|13.655954631379963|172.99432892249527|             529|      0|
|   geraldo perdomo|82.55010416666667|12.076041666666667| 147.0917622523462|             960|      3|
|    zach mckinstry|86.80791366906475|16.610071942446044| 185.5251798561151|             695|      0|
|      heliot ramos|90.97135549872125|10.398976982097187|164.33333333333334|             391|      1|
|     michael busch|89.66396866840729|15.631853785900784|171.84595300261097|             383|      0|
|      brett wisely|84.53505535055349|14.531365313653136|165.07835820895522|      